# Yet Another Tensorflow Tutorial

Currently in-progress

### Prerequisites

This tutorial is built for Tensorflow 1.0 and Python 3.6

### Introduction

Tensorflow is hard to use.

Weaknesses of existing tutorials:
1. Use preloaded example datasets, so it takes time for beginners to figure out how to load your own data
2. Few explanations of what is actually happening

In [16]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [17]:
print(tf.__version__)

1.0.0


#### 0. Loading data

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


The one_hot parameter defines the output as a 1D vector with ten values, where 9 values are zero and 1 value is one, and the location of the one is the identity of the output.

Examples:
    - 0 = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    - 3 = [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
    - 9 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

#### 1. Define placeholder for input: image and label

In [3]:
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

X is a 2D array providing information on the images. Each image is 28x28 pixels, so there are $28 \times 28 = 784$ input values.

y is a 2D array for the predicted outputs. There are ten possible predicted values, the ten digits 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, so there are 10 possible output values.

The ```None``` first dimension means that later, the size of the array can be changed.

#### 2. Define variables for model

In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

W = weight_variable([784, 10])
b = bias_variable([10])

The plan is to build a prediction function of form:
$$y = w \cdot x + b$$

Matrix arithmetic means W should be of dimension 784 x 10 (since X has dimension n x 784) and b should be of dimension n x 10

#### 3. Define Loss function and Optimizer function

In [5]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

- Neural network: regression + softmax
- Loss function: how far off the prediction is from the label
- Optimizer algorithm: tweaks the variables

Softmax: function for assigning probabilities to an object
    - add up
    - sum should be 1.0

cross_entropy is the loss

train_step is the optimizer. Here it uses a Gradient Descent.

#### 4. Accuracy calculation

In [6]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
correct_prediction = tf.cast(correct_prediction, tf.float32)
accuracy = tf.reduce_mean(correct_prediction)

#### 5. Connect to runtime and run graph

In [7]:
sess = tf.InteractiveSession()

In [20]:
for i in range(1100):
    batch = mnist.train.next_batch(1)
    sess.run(train_step, feed_dict={x: batch[0], y_: batch[1]})
    
print("TestAccuracy %g" % sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

TestAccuracy 0.098


Why the error? Variables have not been initialized

#### 6. Initialize variables

In [21]:
tf.global_variables_initializer().run()

In [23]:
for i in range(1100):
    batch = mnist.train.next_batch(1)
    if i % 100 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y_:batch[1]})
        print("Step %d, Training Accuracy %g" % (i, float(train_accuracy)))
    sess.run(train_step, feed_dict={x: batch[0], y_: batch[1]})
    
print("TestAccuracy %g" % sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

Step 0, Training Accuracy 0
Step 100, Training Accuracy 0
Step 200, Training Accuracy 0
Step 300, Training Accuracy 1
Step 400, Training Accuracy 0
Step 500, Training Accuracy 0
Step 600, Training Accuracy 1
Step 700, Training Accuracy 0
Step 800, Training Accuracy 0
Step 900, Training Accuracy 0
Step 1000, Training Accuracy 0
TestAccuracy 0.098


#### 7. Try larger batch

In [24]:
for i in range(1100):
    batch = mnist.train.next_batch(100)
    if i % 100 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y_:batch[1]})
        print("Step %d, Training Accuracy %g" % (i, float(train_accuracy)))
    sess.run(train_step, feed_dict={x: batch[0], y_: batch[1]})
    
print("TestAccuracy %g" % sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

Step 0, Training Accuracy 0.1
Step 100, Training Accuracy 0.08
Step 200, Training Accuracy 0.11
Step 300, Training Accuracy 0.06
Step 400, Training Accuracy 0.1
Step 500, Training Accuracy 0.1
Step 600, Training Accuracy 0.08
Step 700, Training Accuracy 0.1
Step 800, Training Accuracy 0.11
Step 900, Training Accuracy 0.1
Step 1000, Training Accuracy 0.09
TestAccuracy 0.098


#### 8. Add FileWriter to visualize with TensorBoard

In [34]:
LOGDIR = './tesnroflow_logs/mnist_deep'

writer = tf.summary.FileWriter(LOGDIR, sess.graph)

#### 9. Add names and name scope to make it easier to read the graph

In [35]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name='weight')

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name='bias')

x = tf.placeholder(tf.float32, [None, 784], name='x')
y_ = tf.placeholder(tf.float32, [None, 10], name='labels')

In [36]:
with tf.name_scope('softmax'):
    y = tf.nn.softmax(tf.matmul(x, W) + b, name='y')
    
with tf.name_scope('loss'):
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]), name='cross_entropy')
    
with tf.name_scope('optimizer'):
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy, name='train_step')
    
with tf.name_scope('accuracy'):
    accuracy = tf.reduce_mean(correct_prediction, name='accuracy')

#### 10. Viewing images in TensorBoard

In [37]:
with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', x_image, 4)

In [38]:
merged = tf.summary.merge_all()

In [45]:
def weight_variable(shape):
    """Generates a weight variable of a given shape."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name='weight')


def bias_variable(shape):
    """Generates a bias variable of a given shape."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name='bias')

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# Placeholder that will be fed image data.
x = tf.placeholder(tf.float32, [None, 784], name='x')
# Placeholder that will be fed the correct labels.
y_ = tf.placeholder(tf.float32, [None, 10], name='labels')

# Define weight and bias.
W = weight_variable([784, 10])
b = bias_variable([10])

with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', x_image, 4)

# Here we define our model which utilizes the softmax regression.
with tf.name_scope('softmax'):
    y = tf.nn.softmax(tf.matmul(x, W) + b, name='y')

# Define our loss.
with tf.name_scope('loss'):
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]), name='cross_entropy')

# Define our optimizer.
with tf.name_scope('optimizer'):
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy, name='train_step')

# Define accuracy.
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    correct_prediction = tf.cast(correct_prediction, tf.float32, name='correct_prediction')
    accuracy = tf.reduce_mean(correct_prediction, name='accuracy')

# Launch session.
sess = tf.InteractiveSession()

# Initialize variables.
tf.global_variables_initializer().run()

# Merge all the summary data
merged = tf.summary.merge_all()

# Create summary writer
writer = tf.summary.FileWriter(LOGDIR, sess.graph)

# Do the training.
for i in range(1100):
    batch = mnist.train.next_batch(100)
    if i % 5 == 0:
        summary = sess.run(merged, feed_dict={x: batch[0], y_: batch[1]})
        writer.add_summary(summary, i)
    if i % 100 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y_: batch[1]})
        print("Step %d, Training Accuracy %g" % (i, float(train_accuracy)))
    sess.run(train_step, feed_dict={x: batch[0], y_: batch[1]})

# See how model did.
print("Test Accuracy %g" % sess.run(accuracy, feed_dict={x: mnist.test.images,
                                                             y_: mnist.test.labels}))

# Close summary writer
writer.close()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float
	 [[Node: Placeholder = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Placeholder', defined at:
  File "/Users/macbook/anaconda/envs/tensorflow/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/macbook/anaconda/envs/tensorflow/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/macbook/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/macbook/anaconda/envs/tensorflow/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/macbook/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/macbook/anacon

#### 11. View line graphs and histograms of variables

In [42]:
tf.summary.histogram('weight', W)
tf.summary.histogram('bias', b)
tf.summary.histogram('softmax', y)
tf.summary.scalar('loss', cross_entropy)
tf.summary.scalar('accuracy', accuracy)

<tf.Tensor 'accuracy_4:0' shape=() dtype=string>

In [43]:
writer.close()